In [1]:
import argparse
import inspect
import json
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
try:
    from transformers import BitsAndBytesConfig  # type: ignore
except ImportError:  # pragma: no cover - optional optimization
    BitsAndBytesConfig = None

try:
    from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training  # type: ignore[import]
    PEFT_AVAILABLE = True
except ImportError:  # pragma: no cover - dependency guard
    LoraConfig = TaskType = None  # type: ignore[assignment]
    get_peft_model = prepare_model_for_kbit_training = None  # type: ignore[assignment]
    PEFT_AVAILABLE = False

import tqdm.auto as tqdm

/home/jedld/miniconda3/envs/ai351/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Experiment # 4
======================

XLM-RoBERTa (Multilingual) backbone + classifier head

This experiment uses `xlm-roberta-base` (or similar multilingual models) instead of Llama 3. XLM-RoBERTa is an encoder-based model optimized for multilingual representation, making it a strong candidate for Tagalog and mixed-language datasets.


In [2]:
MAX_LENGTH = 512
TRAIN_BF16 = True
TRAIN_FP16 = True
TRAIN_HEAD_ONLY = False # Changed to False to enable LoRA fine-tuning

LORA_R = 16
LORA_ALPHA = 64 # Usually 2x Rank
LORA_DROPOUT = 0.05

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
@dataclass
class TokenizedDataset:
    train: Dataset
    validation: Dataset
    test: Dataset
    label2id: Dict[str, int]
    id2label: Dict[int, str]

In [5]:
def compute_metrics(eval_pred) -> Dict[str, float]:
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]
    preds = np.argmax(logits, axis=1)
    labels = labels.astype(int)
    accuracy = (preds == labels).mean().item()

    classes = np.unique(labels)
    precisions: List[float] = []
    recalls: List[float] = []
    f1_scores: List[float] = []

    for cls in classes:
        tp = np.logical_and(preds == cls, labels == cls).sum()
        fp = np.logical_and(preds == cls, labels != cls).sum()
        fn = np.logical_and(preds != cls, labels == cls).sum()

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    macro_precision = float(np.mean(precisions))
    macro_recall = float(np.mean(recalls))
    macro_f1 = float(np.mean(f1_scores))

    return {
        "accuracy": float(accuracy),
        "precision": macro_precision,
        "recall": macro_recall,
        "f1": macro_f1,
    }

def convert_labels(df: pd.DataFrame, label2id: Dict[str, int]) -> pd.DataFrame:
    mapped = df.copy()
    mapped["label"] = mapped["label"].astype(str).map(label2id)
    if mapped["label"].isnull().any():
        missing = df.loc[mapped["label"].isnull(), "label"].unique()
        raise ValueError(f"Encountered labels not in training set: {missing}")
    mapped["label"] = mapped["label"].astype(int)
    return mapped

def prepare_label_mappings(train_df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[int, str]]:
    unique_labels = sorted({str(label) for label in train_df["label"].tolist()})
    if len(unique_labels) < 2:
        raise ValueError("Classification requires at least two unique labels.")
    label2id = {label: idx for idx, label in enumerate(unique_labels)}
    id2label = {idx: label for label, idx in label2id.items()}
    return label2id, id2label

def load_dataframe(path: Path, text_column: str, label_column: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    if text_column not in df.columns or label_column not in df.columns:
        raise ValueError(f"File {path} must contain '{text_column}' and '{label_column}' columns.")
    return df[[text_column, label_column]].rename(columns={text_column: "text", label_column: "label"})

def log_trainable_parameters(model: torch.nn.Module) -> None:
    total_params = sum(param.numel() for param in model.parameters())
    trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
    ratio = (trainable_params / total_params * 100.0) if total_params > 0 else 0.0
    print(
        f"Trainable params: {trainable_params:,} | Total params: {total_params:,} | Trainable%: {ratio:.4f}"
    )

def tokenize_dataframe(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    tokenizer,
    max_length: int,
) -> TokenizedDataset:
    train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
    val_dataset = Dataset.from_pandas(val_df, preserve_index=False)
    test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

    def tokenize_batch(batch: Dict[str, List[str]]) -> Dict[str, List[int]]:
        return tokenizer(
            batch["text"],
            truncation=True,
            padding=False,
            max_length=max_length,
        )

    train_dataset = train_dataset.map(tokenize_batch, batched=True)
    val_dataset = val_dataset.map(tokenize_batch, batched=True)
    test_dataset = test_dataset.map(tokenize_batch, batched=True)

    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    return TokenizedDataset(train=train_dataset, validation=val_dataset, test=test_dataset, label2id={}, id2label={})


In [6]:
def build_model(
    base_model: str,
    num_labels: int,
    device: str,
    bf16 : bool,
    fp16 : bool,
    train_head_only: bool,
    lora_r: int,
    lora_alpha: int,
    lora_dropout: float,
    lora_target_modules: Optional[List[str]],
) -> torch.nn.Module:
    # Use bfloat16 or float16 to fit in 24GB VRAM (Llama-3-8B is ~16GB in half precision)
    if bf16:
        torch_dtype = torch.bfloat16
    elif fp16:
        torch_dtype = torch.float16
    else:
        torch_dtype = torch.float32

    def quant_config(is_4bit: bool, is_8bit: bool):
        if not (is_4bit or is_8bit):
            return None
        if BitsAndBytesConfig is None:
            raise RuntimeError("bitsandbytes is required for quantized loading but is not available.")
        if is_4bit and is_8bit:
            raise ValueError("Only one of 4-bit or 8-bit quantization can be enabled at a time.")
        if is_4bit:
            return BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16 if bf16 else torch.float16,
            )
        return BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

    # For smaller models like XLM-R (270M), we might not need quantization, but keeping logic generic
    quantization_config = quant_config(False, False)

    device_map = None
    if device == "auto":
        if quantization_config is not None:
            device_map = "auto"
        elif not train_head_only:
            device_map = "auto"

    model = AutoModelForSequenceClassification.from_pretrained(
        base_model,
        num_labels=num_labels,
        torch_dtype=torch_dtype,
        device_map=device_map,
        low_cpu_mem_usage=True,
        quantization_config=quantization_config,
    )
    model.config.problem_type = "single_label_classification"

    if train_head_only:
        for name, param in model.named_parameters():
            trainable = name.startswith("score") or name.startswith("classifier")
            param.requires_grad = trainable
        # Handle different classifier head names (RoBERTa uses 'classifier', Llama uses 'score')
        if hasattr(model, "score") and hasattr(model.score, "weight"):
            torch.nn.init.normal_(model.score.weight, mean=0.0, std=model.config.initializer_range)
            if getattr(model.score, "bias", None) is not None:
                torch.nn.init.zeros_(model.score.bias)
        elif hasattr(model, "classifier"):
             # RoBERTa classifier is a module, usually initialized automatically, but we can ensure grad
             pass

        if device != "auto":
            target_device = torch.device(device)
            model.to(target_device)
        return model

    if not PEFT_AVAILABLE:
        raise ImportError(
            "LoRA fine-tuning requires the 'peft' package. Install it via 'pip install peft'."
        )

    if lora_target_modules is None:
        # Auto-detect target modules based on model type
        if "llama" in base_model.lower():
            target_modules = [
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj",
            ]
        elif "roberta" in base_model.lower() or "bert" in base_model.lower():
            target_modules = ["query", "value"] # Standard for RoBERTa/BERT
        else:
            # Fallback or let PEFT decide (passing None to LoraConfig might work for some, but explicit is safer)
            target_modules = ["q_proj", "v_proj"] 
    else:
        target_modules = lora_target_modules

    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type=TaskType.SEQ_CLS,
    )

    model = get_peft_model(model, lora_config)
    
    # Ensure model is on the correct device when using LoRA
    if device != "auto":
        model.to(device)
        
    model.print_trainable_parameters()
    return model

In [7]:
train_csv =Path("data/combined/processed/train.csv")
val_csv =Path("data/combined/processed/validation.csv")
test_csv=Path("data/combined/processed/test.csv")


In [8]:
train_df = load_dataframe(train_csv, "text", "label")
val_df = load_dataframe(val_csv,  "text", "label")
test_df = load_dataframe(test_csv,  "text", "label")

label2id, id2label = prepare_label_mappings(train_df)
train_df = convert_labels(train_df, label2id)
val_df = convert_labels(val_df, label2id)
test_df = convert_labels(test_df, label2id)


In [9]:
# Using XLM-RoBERTa Large as requested
# Other options: 
# - "xlm-roberta-base" (Faster, less memory)
# - "jcblaise/roberta-tagalog-base" (Tagalog-specific)
base_model = "xlm-roberta-large"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right" # RoBERTa usually handles padding correctly, but right is standard

tokenized = tokenize_dataframe(train_df, val_df, test_df, tokenizer, MAX_LENGTH)
tokenized = TokenizedDataset(
    train=tokenized.train,
    validation=tokenized.validation,
    test=tokenized.test,
    label2id=label2id,
    id2label=id2label,
)

Map:   0%|          | 0/18168 [00:00<?, ? examples/s]

Map:   0%|          | 0/2271 [00:00<?, ? examples/s]

Map:   0%|          | 0/2271 [00:00<?, ? examples/s]

Setup model builder, pretrained XLM Roberta backbone + classifier head.

In [11]:
model = build_model(
        base_model=base_model,
        num_labels=len(label2id),
        device=device,
        bf16=TRAIN_BF16,
        fp16=TRAIN_FP16,
        train_head_only=TRAIN_HEAD_ONLY,
        lora_r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        # Target all linear layers in RoBERTa for maximum adaptation
        lora_target_modules=["query", "key", "value", "dense"] 
    )

model.config.label2id = label2id
model.config.id2label = id2label
if hasattr(model.config, "use_cache"):
    model.config.use_cache = False

if TRAIN_HEAD_ONLY:
    log_trainable_parameters(model)

# DataCollatorWithPadding uses dynamic padding (padding="longest") which is memory efficient
# It pads each batch to the longest sequence in that batch, rather than the model's max length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,129,538 || all params: 568,022,020 || trainable%: 1.4312


In [12]:
# Model Architecture Summary using torchinfo
try:
    from torchinfo import summary
except ImportError:
    print("Installing torchinfo...")
    !pip install torchinfo
    from torchinfo import summary

print(f"Generating model summary for {base_model}...")

# Create dummy inputs on the correct device to handle the forward pass correctly
dummy_batch_size = 1
dummy_inputs = {
    "input_ids": torch.randint(0, tokenizer.vocab_size, (dummy_batch_size, MAX_LENGTH), dtype=torch.long).to(device),
    "attention_mask": torch.ones((dummy_batch_size, MAX_LENGTH), dtype=torch.long).to(device)
}

# Display concise summary
summary(
    model,
    input_data=dummy_inputs,
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
    depth=3, # Depth 3 gives a good balance of detail for Transformer models
)

Generating model summary for xlm-roberta-large...


Layer (type (var_name))                                                          Input Shape          Output Shape         Param #              Trainable
PeftModelForSequenceClassification (PeftModelForSequenceClassification)          --                   [1, 2]               --                   Partial
├─LoraModel (base_model)                                                         --                   [1, 2]               --                   Partial
│    └─XLMRobertaForSequenceClassification (model)                               --                   --                   --                   Partial
│    │    └─XLMRobertaModel (roberta)                                            [1, 512]             [1, 512, 1024]       565,918,720          Partial
│    │    └─ModulesToSaveWrapper (classifier)                                    [1, 512, 1024]       [1, 2]               2,103,300            Partial
Total params: 568,022,020
Trainable params: 8,129,538
Non-trainable params: 559,892,48

Setup training
==============

In [13]:
BATCH_SIZE = 16 
EVAL_BATCH_SIZE = 8
OUTPUT_DIR = Path("models/xlm_roberta_large_classifier")
GRADIENT_ACCUMULATION_STEPS = 4 
EPOCHS = 25 # Increased epochs for LoRA convergence
LEARNING_RATE = 2e-5 # Higher LR is standard for LoRA (vs 1e-5 or 2e-5 for full finetuning)
LR_SCHEDULER_TYPE = "cosine" # Cosine schedule is often better for LoRA
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.06
EVAL_STEPS = 100
LOGGING_STEPS = 50
SAVE_STEPS = 100
SAVE_TOTAL_LIMIT = 3
BF16 = True
FP16 = False

In [14]:
SEED = 42

In [15]:
kwargs: Dict[str, Any] = {}
signature = inspect.signature(TrainingArguments.__init__)
param_names = set(signature.parameters.keys())

def set_kw(name: str, value: Any) -> None:
    if name in param_names:
        kwargs[name] = value

def set_kw_alias(value: Any, *aliases: str) -> None:
    for alias in aliases:
        if alias in param_names:
            kwargs[alias] = value
            return

set_kw("output_dir", str(OUTPUT_DIR))
set_kw("per_device_train_batch_size", BATCH_SIZE)
set_kw("per_device_eval_batch_size", EVAL_BATCH_SIZE)
set_kw("gradient_accumulation_steps", GRADIENT_ACCUMULATION_STEPS)
set_kw("num_train_epochs", EPOCHS)
set_kw("learning_rate", LEARNING_RATE)
set_kw("weight_decay", WEIGHT_DECAY)
set_kw("warmup_ratio", WARMUP_RATIO)
set_kw("lr_scheduler_type", LR_SCHEDULER_TYPE) # Use the defined scheduler type
set_kw("logging_steps", LOGGING_STEPS)
set_kw_alias("steps", "evaluation_strategy", "eval_strategy")
set_kw("eval_steps", EVAL_STEPS)
set_kw_alias("steps", "save_strategy", "checkpointing_strategy")
set_kw("save_steps", SAVE_STEPS)
set_kw("save_total_limit", SAVE_TOTAL_LIMIT)
set_kw("load_best_model_at_end", True)
set_kw("metric_for_best_model", "f1")
set_kw("greater_is_better", True)
set_kw("bf16", BF16)
set_kw("fp16", FP16)
set_kw("push_to_hub", False)
set_kw("hub_model_id", None)
set_kw("hub_token", None)
set_kw("remove_unused_columns", False)
set_kw("seed", SEED)

set_kw("report_to", ["tensorboard"])
set_kw("disable_tqdm", False) # Ensure progress bar is shown

training_args = TrainingArguments(**kwargs)

In [16]:
checkpoints_dir = OUTPUT_DIR / "checkpoints"
checkpoints_dir.mkdir(parents=True, exist_ok=True)
args = {
    "batch_size": BATCH_SIZE
}

trainer_kwargs = {
        "model": model,
        "args": training_args,
        "train_dataset": tokenized.train,
        "eval_dataset": tokenized.validation,
        "data_collator": data_collator,
        "compute_metrics": compute_metrics,
    }
trainer_signature = inspect.signature(Trainer.__init__)
if "tokenizer" in trainer_signature.parameters:
    trainer_kwargs["tokenizer"] = tokenizer
if "processing_class" in trainer_signature.parameters:
    trainer_kwargs["processing_class"] = tokenizer


In [ ]:
import numpy as np

N_RUNS = 5
seeds = [42 + i for i in range(N_RUNS)]
all_metrics = {
    "Train": {"accuracy": [], "precision": [], "recall": [], "f1": []},
    "Validation": {"accuracy": [], "precision": [], "recall": [], "f1": []},
    "Test": {"accuracy": [], "precision": [], "recall": [], "f1": []},
}

print(f"Starting experiment with N={N_RUNS} runs...")

for i, seed in enumerate(seeds):
    print(f"\n" + "="*40)
    print(f"Run {i+1}/{N_RUNS} (Seed: {seed})")
    print("="*40)
    
    # 1. Re-build model to ensure fresh weights
    # We use the global configuration variables
    model = build_model(
        base_model=base_model,
        num_labels=len(label2id),
        device=device,
        bf16=TRAIN_BF16,
        fp16=TRAIN_FP16,
        train_head_only=TRAIN_HEAD_ONLY,
        lora_r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        lora_target_modules=["query", "key", "value", "dense"]
    )
    model.config.label2id = label2id
    model.config.id2label = id2label
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False
        
    # 2. Update Training Arguments with new seed
    # We update the global kwargs dictionary and recreate TrainingArguments
    set_kw("seed", seed)
    set_kw("output_dir", str(OUTPUT_DIR / f"run_{seed}")) 
    training_args = TrainingArguments(**kwargs)
    
    # 3. Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized.train,
        eval_dataset=tokenized.validation,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )
    
    # 4. Train
    trainer.train()
    
    # 5. Evaluate
    for split_name, dataset in [("Train", tokenized.train), ("Validation", tokenized.validation), ("Test", tokenized.test)]:
        metrics = trainer.evaluate(eval_dataset=dataset)
        # Extract metrics (remove 'eval_' prefix)
        all_metrics[split_name]["accuracy"].append(metrics.get("eval_accuracy", 0))
        all_metrics[split_name]["precision"].append(metrics.get("eval_precision", 0))
        all_metrics[split_name]["recall"].append(metrics.get("eval_recall", 0))
        all_metrics[split_name]["f1"].append(metrics.get("eval_f1", 0))

# Calculate and Print Statistics
print("\n" + "="*100)
print(f"RESULTS (N={N_RUNS})")
print(f"{'SPLIT':<15} {'ACCURACY':<25} {'PRECISION':<25} {'RECALL':<25} {'F1':<25}")
print("-" * 100)

for split_name in ["Train", "Validation", "Test"]:
    row_str = f"{split_name:<15} "
    for metric in ["accuracy", "precision", "recall", "f1"]:
        values = all_metrics[split_name][metric]
        mean = np.mean(values)
        std = np.std(values)
        # Format: Mean ± Std
        row_str += f"{mean:.4f} ± {std:.4f}   ".ljust(25)
    print(row_str)
print("="*100)

Starting experiment with N=5 runs...

Run 1/5 (Seed: 42)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,129,538 || all params: 568,022,020 || trainable%: 1.4312


/tmp/ipykernel_4064241/2168342842.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.727700,0.705095,0.476002,0.607285,0.510918,0.352137
200,0.675400,0.657135,0.614707,0.617683,0.601315,0.593904
300,0.649500,0.632120,0.640687,0.641251,0.641989,0.640374
400,0.633800,0.606676,0.664025,0.676276,0.671542,0.663084
500,0.567300,0.522812,0.738882,0.740547,0.741531,0.738792
600,0.513000,0.481516,0.768384,0.767609,0.768883,0.767809
700,0.487200,0.473736,0.769265,0.768866,0.770289,0.768843
800,0.471400,0.459477,0.779833,0.779016,0.777529,0.778094
900,0.444500,0.456780,0.789960,0.789055,0.788070,0.788480
1000,0.457900,0.446561,0.794804,0.793894,0.793037,0.793403


...


Run 3/5 (Seed: 44)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,129,538 || all params: 568,022,020 || trainable%: 1.4312


/tmp/ipykernel_4064241/2168342842.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.749800,0.729856,0.463232,0.231616,0.500000,0.316581
200,0.685900,0.651612,0.623954,0.627797,0.627640,0.623948
300,0.649600,0.628050,0.653016,0.651130,0.651130,0.651130
400,0.630900,0.586248,0.686041,0.696241,0.692701,0.685548
500,0.549000,0.510773,0.751651,0.750330,0.749910,0.750099
600,0.507300,0.494337,0.762219,0.761212,0.759820,0.760342
700,0.493100,0.480373,0.771026,0.771589,0.766981,0.768148
800,0.488000,0.456776,0.783796,0.782661,0.782392,0.782519
900,0.474500,0.458928,0.785117,0.787650,0.780107,0.781678
1000,0.453200,0.479587,0.783796,0.791240,0.789359,0.783729


In [ ]:
# ==========================================
# Training Visualization (Last Run)
# ==========================================
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
except ImportError:
    print("Installing matplotlib and seaborn...")
    !pip install matplotlib seaborn
    import matplotlib.pyplot as plt
    import seaborn as sns

import pandas as pd

# Set style for better aesthetics
sns.set_theme(style="whitegrid")

# Retrieve logs from the trainer (This will be from the LAST run in the loop)
history = trainer.state.log_history
df = pd.DataFrame(history)

# Separate training and evaluation logs
# Training logs usually have 'loss' but no 'eval_loss'
# Eval logs have 'eval_loss'
train_df = df[df['loss'].notna()].dropna(subset=['loss'])
eval_df = df[df['eval_loss'].notna()].dropna(subset=['eval_loss'])

# Create plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Loss Curves
sns.lineplot(ax=axes[0], data=train_df, x='step', y='loss', label='Training Loss', alpha=0.7)
if not eval_df.empty:
    sns.lineplot(ax=axes[0], data=eval_df, x='step', y='eval_loss', label='Validation Loss', marker='o', linestyle='--')

axes[0].set_title('Training and Validation Loss (Last Run)')
axes[0].set_xlabel('Steps')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Plot 2: Metrics
if not eval_df.empty:
    # Select metrics to plot
    metrics = ['eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall']
    available_metrics = [m for m in metrics if m in eval_df.columns]
    
    if available_metrics:
        # Melt dataframe for easier plotting with seaborn
        eval_melted = eval_df.melt(id_vars=['step'], value_vars=available_metrics, var_name='Metric', value_name='Score')
        # Clean up metric names for legend (remove 'eval_' prefix)
        eval_melted['Metric'] = eval_melted['Metric'].str.replace('eval_', '').str.capitalize()
        
        sns.lineplot(ax=axes[1], data=eval_melted, x='step', y='Score', hue='Metric', marker='o')
        axes[1].set_title('Validation Metrics (Last Run)')
        axes[1].set_xlabel('Steps')
        axes[1].set_ylabel('Score')
        axes[1].set_ylim(0, 1.05) # Metrics are usually between 0 and 1
        axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        axes[1].text(0.5, 0.5, 'No standard metrics found in logs', ha='center')
else:
    axes[1].text(0.5, 0.5, 'No validation data available', ha='center')

plt.tight_layout()
plt.show()